# Introduction to Data Science with Julia

<img src="http://julialang.org/images/logo_hires.png" alt="Julia Logo" width="300"></img>

# 目次

- [線形代数](#線形代数)
- [統計量の計算](#統計量の計算)
- [回帰直線](#回帰直線)
- [アンスコムの例](#アンスコムの例)
- [データ分析入門](#データ分析入門)
- [練習問題](#練習問題)

# 線形代数

Julia では線形代数の計算が標準機能として備わっています。

行列 $A$ と行列 $B$ の掛け算は
```julia
A * B
```
と書くだけです。

In [ ]:
A = reshape(1:9, 3, 3)
B = rand(1:10, 3, 3)
@show A
@show B
@show A * B

逆行列や行列式、固有値、固有ベクトルなども簡単に計算することが出来ます。

In [ ]:
@show C = rand(3, 3)
inv(C) # 逆行列

In [ ]:
det(C) # 行列式

In [ ]:
eigvals(C) # 固有値

In [ ]:
eigvecs(C) # 固有ベクトル

In [ ]:
trace(C) # トレース

In [ ]:
rank(C) # ランク

In [ ]:
x = [1, 2, 3]
y = [4, 5, 6]
dot(x,y) # 内積

線形方程式 $Ax = y$ を解く場合には
```julia
A \ y # ￥ はバックスラッシュ
```
とします。行列 $A$ の逆行列は inv(A) として求められるので
```julia
inv(A) * y
```
としても同様の結果が得られますが、A \ y の方がより計算精度が上がります。

In [ ]:
A = rand(3,3)
y = rand(3)
A \ y

その他の行列計算に関しては[公式ドキュメント](http://docs.julialang.org/en/stable/manual/linear-algebra/)を読んでください。

[目次に戻る](#目次)

# 統計量の計算

## 量的データ (numerical data)

Julia では平均や分散などを計算する関数が標準で備わっています。

In [ ]:
# 平均 (mean)
@show x = rand(5)
mean(x)

In [ ]:
# 分散 (variance)
@show x = rand(5)
var(x) # 補正をなくす場合は、 var(x, corrected=false) とする

In [ ]:
# 標準偏差 (standard deviation)
@show x = rand(5)
std(x) # 補正をなくす場合は、 std(x, corrected=false) とする

In [ ]:
# 中央値 (median)
x = 1:5
median(x)

In [ ]:
# 第1四分位点（lower quartile）
quantile(x, 1/4) 

In [ ]:
# 第1四分位点, 中央値, 第3四分位点（upper quartile）
quantile(x, [1/4, 1/2, 3/4]) 

In [ ]:
# 最大値
maximum(x)

In [ ]:
# 最小値
minimum(x)

In [ ]:
# 最小値と最大値を同時に計算
extrema(x)

In [ ]:
# 相関係数
x = 1.0:1.0:12.0
y = x .+ randn(length(x))
cor(x,y)

mean や var などで 2次元配列を引数に取ると要素全体の平均などになりますが、第2引数に 1 と指定すると列ごとの平均、 2 を指定すると行ごとの平均になります。

In [ ]:
x = [1 2 3
    4 5 6 
    7 8 9]
@show mean(x)
@show mean(x, 1) # 列ごとの平均
@show mean(x, 2) # 行ごとの平均

[目次に戻る](#目次)

[StatsBase](http://statsbasejl.readthedocs.io/en/latest/index.html) パッケージを使うと統計量の計算がさらにやりやすくなります。

In [ ]:
using StatsBase

summarystats を使うと、平均と五数要約（[five-number summary](https://en.wikipedia.org/wiki/Five-number_summary)）を一度に計算できます。

In [ ]:
@show x = rand(10)
result = StatsBase.summarystats(x)

In [ ]:
typeof(result)

In [ ]:
fieldnames(result) # 各データへアクセスするための名前

In [ ]:
result.mean

In [ ]:
result.q25

 四分位範囲 (Interquartile Range, IQR) を計算するには iqr を使います。

In [ ]:
StatsBase.iqr(x)

[Z-score](https://en.wikipedia.org/wiki/Standard_score) の計算も出来ます。

In [ ]:
StatsBase.zscore(x)

[目次に戻る](#目次)

## 質的データ (categorical data)

質的データを調べるときには countmap や proportionmap を使うと便利です。
共通要素が何個（何割）あるのかがわかります。

返り値は各要素を key とする辞書です。

In [ ]:
@show x = rand('a':'c', 10)
StatsBase.countmap(x) # 共通要素の個数

In [ ]:
abc = StatsBase.proportionmap(x) # 共通要素の全体の割合

In [ ]:
abc['a']

[目次に戻る](#目次)

## 度数分布

まずは Plots を使ってヒストグラムを書いてみましょう。

In [ ]:
import Plots
Plots.gr(leg=false)

In [ ]:
nd = randn(1000) # 正規分布に従う乱数 1000点
Plots.histogram(nd)

bin を変える場合は
```julia
nbins = 20
```
や
```julia
nbins = -5:0.5:5
```
などとします。分割数を指定する場合は前者で、分割幅を決めたい場合は後者を使うと便利です。

In [ ]:
Plots.histogram(nd, nbins = 20) # 20分割

In [ ]:
Plots.histogram(nd, nbins = -5.0:0.5:5.0) # bin幅を 0.5 にし、-5 〜 5 の範囲でプロット

In [ ]:
Plots.histogram(nd, nbins = -5.0:0.5:5.0, norm=true) # 正規化

Plots を使ってヒストグラムを書けばどのような分布なのかということはわかりますが、各 bin の中に何サンプルあるのかという具体的な数値はわかりません。

具体的な数値を知りたい場合は
```julia
    fit(Histogram, nd, nbins = 20)
    or
    fit(Histogram, nd, -5.0:0.5:5.0)
```
などとします。

結果は
```julia
StatsBase.Histogram{Int64,1,Tuple{FloatRange{Float64}}}
edges:
  -3.5:0.5:3.0
weights: [1,2,13,46,84,149,173,206,164,83,46,27,6]
closed: right
```
のようになります。ここで edges は範囲と分割幅、weights が度数を表します。
各値は 変数名.edges や 変数名.weights をすることで抜き出すことが出来ます。

In [ ]:
ndfreq1 = fit(Histogram, nd, nbins = 20) # 分割数を指定

In [ ]:
ndfreq2 = fit(Histogram, nd, -5.0:0.5:5.0) # 幅と範囲を指定

In [ ]:
ndfreq2.edges

In [ ]:
ndfreq2.weights

求めた結果をヒストグラムと一緒に描写してみます。

In [ ]:
Plots.histogram(nd, nbins = -5.0:0.5:5.0)
x = [(ndfreq2.edges[1][i] + ndfreq2.edges[1][i+1])/2 for i in 1:length(ndfreq2.edges[1])-1] # 
Plots.plot!(x, ndfreq2.weights, marker=:circle)

最頻値 (mode) を求める場合は mode を使います。

In [ ]:
@show a = rand('A':'C', 10)
StatsBase.mode(a)

[目次に戻る](#目次)

# 回帰直線

Julia で回帰直線の切片、傾きを求めるには linreg 関数を使います。

In [ ]:
?linreg

In [ ]:
# y = a + b * x
x = 1.0:12.0
y = [5.5, 6.3, 7.6, 8.8, 10.9, 11.79, 13.48, 15.02, 17.77, 20.81, 22.0, 22.99]
a, b = linreg(x, y)

求めた結果を使ってプロットしてみます。

In [ ]:
Plots.plot(x,y, linetype=:scatter)
Plots.plot!(x, a + b*x)

[目次に戻る](#目次)

# アンスコムの例

データ分析において可視化することがいかに重要かということを知ることの出来る良い例と[アンスコムの例(Anscombe's quartet)](https://en.wikipedia.org/wiki/Anscombe%27s_quartet)というものがあります。

アンスコムの例は4つのデータセットからなり、それぞれのデータセットの平均や分散、回帰直線などはほとんど同じなのに、散布図にすると似ても似つかない分布になるという面白い例です。

アンスコムの例のデータセットは統計ソフト R のデータセットから読み込むことが出来ます。

In [ ]:
using RDatasets
anscombe = RDatasets.dataset("datasets","anscombe")

ここで、読み込んだデータは DataFrame という配列に似たものです。詳細は次回。
まずは、4つのデータをプロットしてみます。

DaatFrame 型は各列を
```julia
anscombe[:X1]
```
のように列名で抜き出すことが出来ます。

In [ ]:
Plots.scatter(anscombe[:X1], anscombe[:Y1], xlims=(0,20))

In [ ]:
Plots.scatter(anscombe[:X2], anscombe[:Y2], xlims=(0,20))

In [ ]:
Plots.scatter(anscombe[:X3], anscombe[:Y3], xlims=(0,20))

In [ ]:
Plots.scatter(anscombe[:X4], anscombe[:Y4], xlims=(0,20))

4つの散布図は似ても似つきません。しかし、統計量は非常に近い値を取ります。

In [ ]:
# Introducing Julia/DataFrames - Wikibooks, open books for an open world 
# https://en.wikibooks.org/wiki/Introducing_Julia/DataFrames#Plotting_Anscombe.27s_Quartet

# print a header
println("Column\tMeanX\tMedianX\tStdDev X\tMeanY\t\t\tStdDev Y\t\tCorr\t")
map((xcol,ycol) -> println(
    xcol,                   "\t",
    mean(anscombe[xcol]),   "\t", 
    median(anscombe[xcol]), "\t", 
    std(anscombe[xcol]),    "\t", 
    mean(anscombe[ycol]),   "\t", 
    std(anscombe[ycol]),    "\t", 
    cor(anscombe[xcol], anscombe[ycol])), 
    
    [:X1, :X2, :X3, :X4], 
    [:Y1, :Y2, :Y3, :Y4]);

[目次に戻る](#目次)

# 練習問題

## 1.

線形方程式<br>
$
\begin{align}
  \left\{
    \begin{array}{l}
      x + 2y = -1 \\
      3x + y = 2
    \end{array}
  \right.
\end{align}
$
を解け

## 2.

以下のコード
```julia
    srand(1)
    scores = rand(0:100, 100, 3)
```
を実行し、配列 scores の
1. 各列の合計を求めよ。
1. 各列の平均を求めよ。
1. 各列の分散を求めよ。

## 3.

次のコードを実行するとタイタニックの乗客乗員の情報を読み込むことが出来る。
```julia
using RDatasets
titanic = RDatasets.dataset("COUNT", "titanic")
```
|   | Survived | Age | Sex | Class |
|---|----------|-----|-----|-------|
| 1 | 1        | 1   | 1   | 1     |
| 2 | 2        | 1   | 1   | 1     |
| 3 | 3        | 1   | 1   | 1     |
| 4 | 4        | 1   | 1   | 1     |
| 5 | 5        | 1   | 1   | 1     |
| 6 | 6        | 1   | 1   | 1     |


ここで各列の数字の意味は以下のとおりである。<br>
Survived<br>
    1=survived; 0=died

Age<br>
    1=adult; 0=child

Sex<br>
    1=Male; 0=female

Class<br>
    ticket class 1= 1st class; 2= second class; 3= third class
    
1. 生存者と死者の数をそれぞれ調べよ。
1. 男女別に生存者と死者の数をそれぞれ調べよ。
1. 年齢別に生存者と死者の数をそれぞれ調べよ。
1. チケットの階級別に生存者と死者の数をそれぞれ調べよ。
1. 1〜4 の結果を図示せよ。

In [ ]:
using RDatasets
titanic = RDatasets.dataset("COUNT", "titanic")
head(titanic)

## 4.

福岡市の月ごとの降水量を調べ棒グラフで図示せよ。

[目次に戻る](#目次)